# News of the Day

This notebook is a modified version of the example found on the Unstructured.IO documentation [site](https://github.com/Unstructured-IO/unstructured/blob/main/examples/chroma-news-of-the-day/news-of-the-day.ipynb). The intent is to show how the used example can be easily integrated with the services offered by [watsonx](https://www.ibm.com/watsonx) to create a robust RAG pipeline.

In this notebook, we'll show how to use [Unstructured.IO](https://unstructured.io/), [ChromaDB](https://www.trychroma.com/), [LangChain](https://github.com/langchain-ai/langchain), [HugginFace]() and IBM [watsonx](https://www.ibm.com/watsonx) to summarize topics from the front page of CNN Lite and use them as context for a grounded LLM interaction.

In order to get more relevant answers we will retrieve the full set of the latest news from the site and then run a specific implementation of the retriever named [Contextual Compression](https://python.langchain.com/docs/modules/data_connection/retrievers/contextual_compression). Instead of querying the similar documents to the question being asked as-is, we ask the LLM to compress them using the context of the given question, so that only the relevant information is used for the LLM generation of the answer.

## Document printing helper

We start off by defining a print helper function. This will come handy throughout the execution in order to examine the documents extracted from the site and how they will we modified.

In [2]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}: \n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

## STEP 1: Gather links with `unstructured`

First, we'll gather links from the [CNN Lite](https://lite.cnn.com/) homepage using the `partition_html` function from `unstructured`. When `unstructured` partitions HTML pages, links are included in the metadata for each element, make link collection a simple task. 

**NOTE: if you are on a Mac and when calling partition_html you encounter the following error message:**

`[nltk_data] Error loading url: <urlopen error [SSL:`

`[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:`

`[nltk_data]     unable to get local issuer certificate (_ssl.c:992)>`

You might want to try issuing the following command on a terminal (you should point to the Python version you are using):
    
`sh "/Applications/Python 3.10/Install Certificates.command"`


In [3]:
from unstructured.partition.html import partition_html
cnn_lite_url = "https://lite.cnn.com/"
elements = partition_html(url=cnn_lite_url)

links = []

for element in elements:
    if element.metadata.link_urls:
        relative_link = element.metadata.link_urls[0][1:]
        if relative_link.startswith("2024"):
            links.append(f"{cnn_lite_url}{relative_link}")

print(f"We retrieved {len(links)} links to documents from {cnn_lite_url}")

We retrieved 97 links to documents from https://lite.cnn.com/


## Ingest individual articles with `UnstructuredURLLoader`

Now that we have the links, we can preprocess individual news articles with `UnstructuredURLLoader`. `UnstructuredURLLoader` fetches content from the web and then uses the `unstructured` `partition` function to extract content and metadata. In this example we preprocess HTML files, but it works with other response types such as `application/pdf` as well. After calling `.load()`, the result is a list of `langchain` `Document` objects.

**NOTE: when calling UnstructuredURLLoader, the necessary binary packages for running the python libmagic library must be present in our OS.** 
If you encounter an error related to missing libmagic packages, try to do the following:

#### Windows

Uninstall preinstalled python-magic libraries:

    pip uninstall python-libmagic
    pip uninstall python-magic 

Install python-magic-bin instead:

    pip install python-magic-bin

#### MAC

Install libmagic with MacPorts from a terminal:

    sudo port install libmagic

If you don't have MacPorts installed on your Mac, install it from the official site: https://www.macports.org/install.php.


In [4]:
from langchain.document_loaders import UnstructuredURLLoader
loaders = UnstructuredURLLoader(urls=links[:200], show_progress_bar=True)

docs = loaders.load()

100%|██████████| 97/97 [00:21<00:00,  4.48it/s]


In [5]:
print(f"We retrieved {len(docs)} links to documents from {cnn_lite_url}")

docs[0]

We retrieved 97 links to documents from https://lite.cnn.com/


Document(page_content='CNN\n\n4/30/2024\n\nWorld\n    /\n\nAfrica\n\nScores missing as Kenya ravaged by mass flooding\n\nBy Larry Madowo and Isaac Oyombe, CNN\n\nUpdated: \n        11:23 AM EDT, Tue April 30, 2024\n\nSource: CNN\n\nAt least 91 people are missing after heavy flooding across Kenya’s capital, Nairobi, the government said Tuesday, after weeks of heavy rain and flash floods ravaged parts of the country.\n\nSome 76 people are missing after flooding near the hard hit town of Mai Mahiu, northwest of Nairobi. Locals and first responders told CNN the disaster was caused by water blowing through a clogged tunnel under a railway bridge. So far, 71 people have been confirmed dead as a result of the incident.\n\nA further 10 people are missing in\xa0Kenya’s eastern region, four in Nairobi County and one in the coastal region, which borders the Indian Ocean, said Isaac Mwaura, a government spokesperson.\n\nThe floods have also led to the displacement of 190,942\xa0Kenyans, which Mwau

## Generate embeddings for the extracted articles

Before loading our knowledge base into a suitable vector DB, we need to generate embeddings for our articles.
Embeddings are a type of transformation that helps computers understand the meaning of words and phrases in a text by converting them into a continuous vector space. This makes it easier for computers to learn and make sense of complex relationships between different concepts.

In generative AI pipelines, embeddings are essential because they allow models to capture the semantic meaning of text data. By mapping words and phrases to vectors, embeddings help models maintain context across sentences and documents, making it possible to generate new text that is both coherent and relevant.

To generate embeddings for our articles, we use IBM_SLATE_30M_ENG model from watsonx.ai model library.

N.B. In order to run this notebook you need to set 3 environment variables, related to your watsonx.ai instance:
- <b>WATSONX_URL</b>: it is a URL with this format "https://{region}.ml.cloud.ibm.com"
- <b>WATSONX_API_KEY</b>: API KEY from your IBM Cloud account. A detailed procedure on how to create an API KEY can be found in the link provided at the end of this cell.
- <b>PRJ_ID</b>: it is the ID of the project created on watsonx.ai platform to run this notebook

Information on how to find/create these variables can be found here: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-credentials.html?context=wx&audience=wdp.

In [6]:
from os import environ

credentials = {
    "url": environ.get("WATSONX_URL"),
    "apikey": environ.get("WATSONX_API_KEY")
}
project_id = environ.get("PRJ_ID")

#from ibm_watsonx_ai.foundation_models.utils import get_embedding_model_specs
#get_embedding_model_specs(credentials.get('url'))

from langchain_ibm import WatsonxEmbeddings
from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes

embeddings = WatsonxEmbeddings(
    model_id=EmbeddingTypes.IBM_SLATE_30M_ENG.value,
    url=credentials["url"],
    apikey=credentials["apikey"],
    project_id=project_id
    )

## Load documents into ChromaDB

With the documents preprocessed and vectorized, we're now ready to load them into ChromaDB. We easily accomplish that leveraging the Chroma integration within Langchain. Once the documents are in Chroma, we can perform a similarity search to retrieve documents related to our topic of interest. Here we choose to limit the retrived documents to 5.

In [7]:
# Split the documents into chunks
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

from langchain.vectorstores.chroma import Chroma
vectorstore = Chroma.from_documents(texts, embeddings)

Created a chunk of size 1034, which is longer than the specified 1000


Created a chunk of size 1061, which is longer than the specified 1000


We are now ready to receive a question form the user and store it the variable named text

In [8]:
#text = input("Type what you want to search:")
text = "What happened during the 2024 NFL Draft?"
print(text)

What happened during the 2024 NFL Draft?


We now perform a first similarity search and retrive the 10 most relevant articles and print them out. We expect this search to return documents that are not directly related to the question that the user proposed to the pipeline. We are only sorting the returned documents based on their  distance.

In [9]:
query_docs = vectorstore.similarity_search(text, k=10)

pretty_print_docs(query_docs)

Document 1: 

CNN

4/30/2024

What we learned during the 2024 NFL Draft

By George Ramsay, CNN

Updated: 
        7:55 AM EDT, Mon April 29, 2024

Source: CNN

The 2024 NFL Draft drew to a close on Saturday as 257 players were selected by teams from across the league, all hopeful of a bright career lying before them.

A record 700,000 people attended this year’s draft in the city of Detroit, including 275,000 on the first day, beating the previous record of 600,000 set in Nashville five years ago.

And those in attendance on Thursday saw an unprecedented interest in quarterbacks and offensive players generally.

Between the Chicago Bears selecting USC QB Caleb Williams with the first pick and the New York Jets making Alabama safety Jaylen Key the latest “Mr. Irrelevant” as the 257th pick, here’s what we learned from this year’s draft.

Teams go QB crazy

For the first time in NFL history, five quarterbacks were selected with the first 10 picks of the draft and six with the first 12.
--

## Compress and Summarize the Documents

After retrieving relevant documents from Chroma, we're ready to compress and then summarize them! There are multiple ways to accomplish this in `langchain`, but `ContextualCompressionRetriever` and `load_summarization_chain` is quite straightforward. In this case we're going to use the IBM Granite model within the available Langchain wrapper so to easily integrate it with our summarization chain. Here we limit the summary to snippets related to our topic of choice.

In order to use IBM Granite model within watsonx, we need to set up all the needed parameters as well as to provide a valid API key and a Project ID.

In [10]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from langchain_ibm import WatsonxLLM

granite = WatsonxLLM(
    model_id='ibm/granite-13b-chat-v2',
    url=credentials["url"],
    apikey=credentials["apikey"],
    project_id=project_id,
    params= {
        GenParams.DECODING_METHOD: DecodingMethods.SAMPLE.value,
        GenParams.MAX_NEW_TOKENS: 1024,
        GenParams.MIN_NEW_TOKENS: 1,
        GenParams.TEMPERATURE: 0.5,
        GenParams.TOP_K: 50,
        GenParams.TOP_P: 1
    }
)

Here we compress the found documents by leveraging the LLM again before passing them into the chain for geneating the answer

In [11]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor   

compressor = LLMChainExtractor.from_llm(granite)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=vectorstore.as_retriever())

compressed_docs = compression_retriever.get_relevant_documents(text)
pretty_print_docs(compressed_docs)

/Users/mrinalduzzi/Desktop/Projects/RAG-with-watsonx/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
/Users/mrinalduzzi/Desktop/Projects/RAG-with-watsonx/.venv/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/mrinalduzzi/Desktop/Projects/RAG-with-watsonx/.venv/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/mrinalduzzi/Desktop/Projects/RAG-with-watsonx/.venv/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output p

Document 1: 

Teams go QB crazy
For the first time in NFL history, five quarterbacks were selected with the first 10 picks of the draft and six with the first 12.

This part of the context is relevant to the question because it highlights a significant trend during the 2024 NFL Draft: the high demand for quarterbacks and offensive players. This information can help answer the question about what happened during the draft by providing context on the major events and trends that occurred during the event.
----------------------------------------------------------------------------------------------------
Document 2: 

* The truth is the whole league had no idea this was coming
* We got no heads up. Kirk got a call from the Falcons when they were on the clock
* After Nix, South Carolina’s Spencer Rattler was the next quarterback to be selected in the draft at No. 150 overall by the New Orleans Saints
* The 137 selections between Thursday’s first round and Saturday’s fifth round set a reco

Now we need to set up our summarization chain: the basic chain types are either "stuff" (i.e. documents are provided as context in a single prompt that is passed to the LLM) or "map-reduce" (i.e. documents are processed in a map-reduce fashion in order to obtain summaries from each single document and provide these summaries as context to the LLM). 

In order to avoid possible limits in the number of token processed by the LLM, we could opt for a map-reduce chain (more information on summarization in langchain can be found at https://python.langchain.com/docs/use_cases/summarization) since we applied compression we can stick with the "stuff" type.

We also define a prompt to pass down the invocation along witht he resulting input documents from the retriever and the compressor.

In [12]:
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(granite, chain_type="stuff")

input = {
    "prompt" : "You are a AI language model designed to function as a specialized Retrieval Augmented Generation (RAG) assistant. When generating responses, prioritize correctness, i.e., ensure that your response is correct given the context and user query, and that it is grounded in the context. Furthermore, make sure that the response is supported by the given document or context. When the question cannot be answered using the context or document, output the following response: 'I'm sorry, I don't know.' Always make sure that your response is relevant to the question. If an explanation is needed, first provide the explanation or reasoning, and then give the final answer.",
    "input_documents" : compressed_docs
}
print(chain.invoke(input)['output_text'])



The 2024 NFL Draft saw a record-breaking 13 quarterbacks selected in the first three rounds, with five going in the first 10 picks. The demand for quarterbacks and offensive players was unprecedented, leading to a significant trend in the draft. Offensive linemen, in particular, saw a record 55 selected, with 28 being offensive tackles. Marvin Harrison Jr., son of Pro Football Hall of Famer Marvin Harrison, was selected fourth overall by the Arizona Cardinals. Xavier Worthy, who set a record for the fastest 40-yard dash in the NFL Combine, was selected by the Kansas City Chiefs with the final pick of the first round.


Check out IBM _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts.